# Lab | Data Aggregation and Filtering

In [18]:
import pandas as pd

customer = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv")

display(customer.columns)

display(customer.head())

display(customer.shape)

Index(['Unnamed: 0', 'Customer', 'State', 'Customer Lifetime Value',
       'Response', 'Coverage', 'Education', 'Effective To Date',
       'EmploymentStatus', 'Gender', 'Income', 'Location Code',
       'Marital Status', 'Monthly Premium Auto', 'Months Since Last Claim',
       'Months Since Policy Inception', 'Number of Open Complaints',
       'Number of Policies', 'Policy Type', 'Policy', 'Renew Offer Type',
       'Sales Channel', 'Total Claim Amount', 'Vehicle Class', 'Vehicle Size',
       'Vehicle Type'],
      dtype='object')

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN
1,1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,1/17/11,Medical Leave,F,...,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,NaN


(10910, 26)

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

In [28]:
# Data cleaning:

# display(customer.isnull().sum())

# customer_dropnulls=customer.dropna()

# customer_dropnulls.shape

# drop unnamed column

customer.drop("Unnamed: 0",axis=1,inplace=True)

# 1) replace unknown state with NA as we don't want to lose these rows

customer["State"].fillna("NA",inplace=True)

# 2) replace nulls for response with No (logical replacement)

customer["Response"].fillna("No",inplace=True)

# 3) Fill months since last claim with NA if unknown

customer["Months Since Last Claim"].fillna("NA",inplace=True)

# 4) Fill number of open complaints with 0 if null

customer["Number of Open Complaints"].fillna(0,inplace=True)

# 5) Vehicle class, size, type - also fill with NA so we won't lose these rows

customer["Vehicle Class"].fillna("NA",inplace=True)
customer["Vehicle Size"].fillna("NA",inplace=True)
customer["Vehicle Type"].fillna("NA",inplace=True)

display(customer.isnull().sum())

Customer                         0
State                            0
Customer Lifetime Value          0
Response                         0
Coverage                         0
Education                        0
Effective To Date                0
EmploymentStatus                 0
Gender                           0
Income                           0
Location Code                    0
Marital Status                   0
Monthly Premium Auto             0
Months Since Last Claim          0
Months Since Policy Inception    0
Number of Open Complaints        0
Number of Policies               0
Policy Type                      0
Policy                           0
Renew Offer Type                 0
Sales Channel                    0
Total Claim Amount               0
Vehicle Class                    0
Vehicle Size                     0
Vehicle Type                     0
dtype: int64

In [30]:
customer=customer.rename(columns={col: col.replace(" ","_").lower() for col in customer.columns})
customer.columns

Index(['customer', 'state', 'customer_lifetime_value', 'response', 'coverage',
       'education', 'effective_to_date', 'employmentstatus', 'gender',
       'income', 'location_code', 'marital_status', 'monthly_premium_auto',
       'months_since_last_claim', 'months_since_policy_inception',
       'number_of_open_complaints', 'number_of_policies', 'policy_type',
       'policy', 'renew_offer_type', 'sales_channel', 'total_claim_amount',
       'vehicle_class', 'vehicle_size', 'vehicle_type'],
      dtype='object')

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

In [38]:
#customer1=customer[['total_claim_amount'>1000 & 'response'=="Yes"]]
customer.dtypes

customer1=customer[(customer.total_claim_amount > 1000) & (customer.response =="Yes")]

customer1.shape

(67, 25)

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

In [41]:
customer2=customer[customer.response =="Yes"]

customer2.groupby(['policy_type', 'gender'])['total_claim_amount'].agg(['mean'])

# Conclusions:

# for corporate auto and special auto, average claim is higher for females than males
# opposite true for personal auto

mean
policy_type    gender            
Corporate Auto F       433.738499
               M       408.582459
Personal Auto  F       452.965929
               M       457.010178
Special Auto   F       453.280164
               M       429.527942

In [43]:
# pivot table

pivot_customer2 = customer2.pivot_table(index='policy_type', columns='gender', values=['total_claim_amount'], aggfunc='mean')
pivot_customer2

total_claim_amount            
gender                          F           M
policy_type                                  
Corporate Auto         433.738499  408.582459
Personal Auto          452.965929  457.010178
Special Auto           453.280164  429.527942

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [48]:
result=customer.groupby(['state'])['customer'].agg(['count'])
result_filtered=result[result['count'] >= 500]
result_filtered

,count
state,
Arizona,1937
California,3552
NA,631
Nevada,993
Oregon,2909
Washington,888


4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [49]:
customer.groupby(['education', 'gender'])['customer_lifetime_value'].agg(['max','min','median'])

max          min       median
education            gender                                       
Bachelor             F       73225.95652  1904.000852  5640.505303
                     M       67907.27050  1898.007675  5548.031892
College              F       61850.18803  1898.683686  5623.611187
                     M       61134.68307  1918.119700  6005.847375
Doctor               F       44856.11397  2395.570000  5332.462694
                     M       32677.34284  2267.604038  5577.669457
High School or Below F       55277.44589  2144.921535  6039.553187
                     M       83325.38119  1940.981221  6286.731006
Master               F       51016.06704  2417.777032  5729.855012
                     M       50568.25912  2272.307310  5579.099207

In [ ]:
# max lifetime value from males with high school or below
# min lifetime value from females with a bachelors degree
# the median is around 5300 to 6200 for all categories
# further analysis required to better understand the relationship between education level and value, but it does appear that 
# those with highschool or below may be the most valuable clients 

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
# your code goes here